In [4]:
from bs4 import BeautifulSoup as bs
import requests
import os

In [16]:
class Crawler:
    def __init__(self,base_url,store_dir='./store'):
        self.url_frontier = []
        self.base_url = base_url
        if not os.path.exists(store_dir):
            os.mkdir(store_dir)
        self.store = store_dir
    
    def scrap(self, seed_url, num_docs=100):
        
        # URL Frontier 
        self.url_frontier = []
        self.url_frontier.append(seed_url)
        scraped_links = set()
        
        # Doc Scraped Count
        cnt = 0
        
        # Crawling process
        while cnt<num_docs and not len(self.url_frontier)==0:
            url = self.base_url + self.url_frontier[0]
            self.url_frontier.pop(0)
            
            if url in scraped_links:
                continue
            
            scraped_links.add(url)
            
            
            file_path = self.store + "/" + str(cnt) + ".txt"
            file = open(file_path, 'w')
            
            page = requests.get(url)
            
            content,links = self._parse(page)
            
            self._add_links(links)
            
            file.write(self._to_text(content))
            file.close()
            
            cnt += 1
        
    def _to_text(self,content):
        text = "Title : "+ content['title']
        text += "\n\n"
        text += "\n".join(content["content"])
        return text
    
    def _parse(self,page):
        soup = bs(page.content, 'html.parser')
        
        body = soup.find(id="mw-content-text").find('div')
        
        title = soup.find(id="firstHeading")
        table_of_contents = soup.find(id="toc")
        content = body.find_all(['h2','h3', 'h4', 'p', 'ul'])
        
        text = {}
        
        text["title"] = title.get_text()
        text["content"], links = self._parse_content(content)
        
        return (text,links)
    
    def _parse_toc(self,toc):
        raw_list = toc.get_text().split("\n")
        tuple_list = [ t.split(" ") for t in raw_list]
        
        final = []
        for n,head in tuple_list:
            if n.find('.') == -1:
                final.push(head)
                
        return final
                
    
    def _parse_content(self,content):
        text = []
        links = []
        for tag in content :
            text.append(tag.get_text())
            links.extend(tag.find_all('a'))
        
        return (text,links)
    
    def _add_links(self,links):
        flag = True
        for link in links:
            s = str(link)
            k = s.find('>')
            s = s[1:k].split(" ")
            for t in s:
                if t.find("href") != -1:
                    url = t.split("=")[1][1:-1]
                    if url[:6] == "/wiki/" and url[6:].find('/') == -1:
                        self.url_frontier.append(url)
            
        

In [17]:
wiki = Crawler("https://en.wikipedia.org")
wiki.scrap("/wiki/Corona",100)